# ResMLP
#### Feedforward networks for image classification written in PyTorch

### Import and install extra libraries



In [ ]:
import torch
import torch.backends.cudnn as cudnn

!pip install torch_optimizer einops
import torch_optimizer as optim

import dataset
from model import ResMLP
import learning_utils

### Setting device (CPU or GPU)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

### Build ResMLP model

In [ ]:
model = ResMLP(in_channels=3, image_size=96, patch_size=16, num_classes=10, dim=384, depth=12, mlp_dim=384*4).to(device)
if device == 'cuda':
        model = torch.nn.DataParallel(model)
        cudnn.benchmark = True

### Hyperparameters


In [ ]:
learning_rate = 5e-3
weight_decay = 0.2

### Loss and optimizer

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Lamb(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

### Execution

In [ ]:
num_epochs = 300
for t in range(num_epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    learning_utils.train(dataset.train_dataloader, model, device, loss_fn, optimizer)
    learning_utils.test(dataset.test_dataloader, model, device, loss_fn)
print('Done!')

### Save the model


In [ ]:
torch.save(model.state_dict(), 'ResMLP_model.pth')